# Building a dashboard to plan a marketing campaign leveraging CARTO data Observatory

Combining different data sources to identify some patterns or understand some behavior in a specific location is a very typical use case in Spatial Data Science.

In this notebook, we will build a dashboard combining different data from CARTO's Data Observatory to help identify the locations with specific characteristics described below.

**Note** this use case leverages premium datasets from [CARTO Data Observatory](https://carto.com/spatial-data-catalog/).


## Use case description

A pharmaceutical lab wants to launch a new marketing campaign to promote a new line of personal care products for senior people in the city of Philadelphia, PA. They know their target group is characterized by:
 - People over 60
 - Medium-high to high income
 - High expenditure in personal care products and services


Given these characteristics, they would like to know which pharmacies and drug stores in the city of Philadelphia they should focus their efforts on.

In order to identify the target drug stores and pharmacies, we will follow the following steps:
 - [Get all pharmacies in Philadelphia](#section1)
 - [Calculate their cathment areas using isochrones](#section2)
 - [Enrich the isochrones with demographic, POI's, and consumption data](#section3)
 - [Build the dashboard to help identify the pharmacies where the campaign can be more successful given the characteristics of the population within their catchment area](#section4)
 


## 0. Setup

Import the packages we'll use.

In [1]:
import geopandas as gpd
import pandas as pd

from cartoframes.auth import set_default_credentials
from cartoframes.data.services import Isolines
from cartoframes.data.observatory import *
from cartoframes.viz import *
from shapely.geometry import box

pd.set_option('display.max_columns', None)

In order to be able to use the Data Observatory via CARTOframes, you need to set your CARTO account credentials first.

Please, visit the [Authentication guide](https://carto.com/developers/cartoframes/guides/Authentication/) for further detail.

In [2]:
username = 'XXXXXXXXXX'
api_key  = 'XXXXXXXXXX'    # Master API key. Do not make this file public!

set_default_credentials(username=username, api_key=api_key)

**Note about credentials**

For security reasons, we recommend storing your credentials in an external file to prevent publishing them by accident when sharing your notebooks. You can get more information in the section _Setting your credentials_ of the [Authentication guide](https://carto.com/developers/cartoframes/guides/Authentication/).

<a id='section1'></a>
## 1. Download all pharmacies in Philadelphia from the Data Observatory

Below is the bounding box of the area of study.

In [3]:
dem_bbox = box(-75.229353,39.885501,-75.061124,39.997898)

We can get the pharmacies from [Pitney Bowes' Consumer Points of Interest](https://carto.com/spatial-data-catalog/browser/dataset/pb_consumer_po_62cddc04/) dataset. This is a premium dataset, so we first need to check that we are subscribed to it.

Take a look at [this template](**LINK to template**) for more details on how to access and download a premium dataset.

In [4]:
Catalog().subscriptions().datasets.to_dataframe()

,slug,name,description,category_id,country_id,data_source_id,provider_id,geography_name,geography_description,temporal_aggregation,time_coverage,update_frequency,is_public_data,lang,version,category_name,provider_name,geography_id,id
0,ags_sociodemogr_f510a947,Sociodemographics - United States of America (...,Census and ACS sociodemographic data estimated...,demographics,usa,sociodemographics,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2019-01-01, 2020-01-01)",yearly,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_sociodemographics_us...
1,pb_points_of_i_94bda91b,Points Of Interest - Spain (Latitude/Longitude),Point of interest database per categories,points_of_interest,esp,points_of_interest,pitney_bowes,Latitude/Longitude - Spain,Location of Points of Interest,monthly,None,monthly,False,eng,v1,Points of Interest,Pitney Bowes,carto-do.pitney_bowes.geography_esp_latlon_v1,carto-do.pitney_bowes.pointsofinterest_pointso...
2,sg_social_dist_667d8e8e,Social Distancing Metrics - United States of A...,"Due to the COVID-19 pandemic, people are curre...",covid19,usa,social_distancing,safegraph,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,daily,None,daily,False,eng,v1,Covid-19,SafeGraph,carto-do-public-data.carto.geography_usa_block...,carto-do.safegraph.covid19_socialdistancing_us...
3,mbi_consumer_pr_c4d89dd7,Consumer Profiles - Japan (Cho-Cho-Moku),The MB International Consumer Styles describe ...,demographics,jpn,consumer_profiles,mbi,Cho-Cho-Moku - Japan,MBI Digital Boundaries for Japan at Cho-Cho-Mo...,yearly,"[2019-01-01, 2020-01-01)",yearly,False,eng,2020,Demographics,Michael Bauer International,carto-do.mbi.geography_jpn_chochomoku_2020,carto-do.mbi.demographics_consumerprofiles_jpn...
4,u360_sociodemogr_28e93b81,Sociodemographics - Spain (Grid 100m),Curation of socio-demographic and socio-econom...,demographics,esp,sociodemographics,unica360,Grid 100m,100x100m grid,yearly,None,yearly,False,eng,2019,Demographics,Unica360,carto-do.unica360.geography_esp_grid100x100m_2019,carto-do.unica360.demographics_sociodemographi...
5,ags_consumer_sp_dbabddfb,Consumer Spending - United States of America (...,The Consumer Expenditure database consists of ...,demographics,usa,consumer_spending,ags,Census Block Group - United States of America,None,yearly,None,yearly,False,eng,2020,Demographics,Applied Geographic Solutions,carto-do.ags.geography_usa_blockgroup_2015,carto-do.ags.demographics_consumerspending_usa...
6,pb_consumer_po_62cddc04,Points Of Interest - Consumer - United States ...,Consumer Point of interest database per catego...,points_of_interest,usa,consumer_points_of_interest,pitney_bowes,Latitude/Longitude - United States of America,Location of Points of Interest,monthly,None,monthly,False,eng,v1,Points of Interest,Pitney Bowes,carto-do.pitney_bowes.geography_usa_latlon_v1,carto-do.pitney_bowes.pointsofinterest_consume...
7,mc_geographic__7980c5c3,Geographic Insights - United States of America...,"Geographic Insights validate, evaluate and ben...",financial,usa,geographic_insights,mastercard,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,monthly,"[2019-01-01, 2020-01-01)",monthly,False,eng,v1,Financial,Mastercard,carto-do-public-data.carto.geography_usa_block...,carto-do.mastercard.financial_geographicinsigh...
8,spa_geosocial_s_d5dc42ae,Geosocial Segments - United States of America ...,"By analysing feeds from Twitter, Instagram, Me...",behavioral,usa,geosocial_segments,spatial_ai,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,quarterly,"[2020-01-01, 2020-04-01)",quarterly,False,eng,v1,Behavioral,Spatial.ai,carto-do-public-data.carto.geography_usa_block...,carto-do.spatial_ai.behavioral_geosocialsegmen...
9,mc_consumer_j

### Download and explore sample

Pitney Bowes POI's are hierarchically classified (levels: trade division, group, class, sub class). 

Since we might not know which level can help us identify all pharmacies, we can start by downloading a sample for a smaller area to explore the dataset. For calculating the bounding box we use [bboxfinder](http://bboxfinder.com).

We start by selecting our dataset and taking a quick look at its first 10 rows.

In [5]:
dataset = Dataset.get('pb_consumer_po_62cddc04')

In [6]:
dataset.head()

,HTTP,ISO3,NAME,SIC1,SIC2,SIC8,CLASS,EMAIL,GROUP,PB_ID,STABB,geoid,FAXNUM,MICODE,TEL_NUM,do_date,LATITUDE,OPEN_24H,POSTCODE,do_label,AREANAME1,AREANAME2,AREANAME3,AREANAME4,BRANDNAME,GEORESULT,LONGITUDE,SUB_CLASS,AGENT_CODE,TRADE_NAME,YEAR_START,STATUS_CODE,BUSINESS_LINE,CURRENCY_CODE,EMPLOYEE_HERE,EXCHANGE_NAME,TICKER_SYMBOL,EMPLOYEE_COUNT,FAMILY_MEMBERS,FRANCHISE_NAME,HIERARCHY_CODE,PARENT_ADDRESS,PARENT_COUNTRY,TRADE_DIVISION,ADDRESSLASTLINE,CONFIDENCE_CODE,MAINADDRESSLINE,PARENT_POSTCODE,FORMATTEDADDRESS,PARENT_AREANAME1,PARENT_AREANAME3,SIC8_DESCRIPTION,ALT_INDUSTRY_CODE,LEGAL_STATUS_CODE,SALES_VOLUME_LOCAL,COUNTRY_ACCESS_CODE,PARENT_BUSINESS_NAME,SUBSIDIARY_INDICATOR,PARENT_STREET_ADDRESS,GLOBAL_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_COUNTRY,SALES_VOLUME_US_DOLLARS,GLOBAL_ULTIMATE_POSTCODE,DOMESTIC_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_INDICATOR,DOMESTIC_ULTIMATE_POSTCODE,DOMESTIC_ULTIMATE_AREANAME1,DOMESTIC_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_BUSINESS_NAME,GLOBAL_ULTIMATE_STREET_ADDRESS,DOMESTIC_ULTIMATE_BUSINESS_NAME,DOMESTIC_ULTIMATE_STREET_ADDRESS
0,None,USA,ONO,None,None,53310000,CAMPS AND RECREATIONAL VEHICLE PARKS,None,"EXECUTIVE, LEGISLATIVE AND GENERAL GOVERNMENT,...",1243815016,CA,2204806297#144.7436111#13.2777778,None,10050191,(402) 933-7829,2020-08-01,39.850845,None,None,LOLLIPOSH CLOTHING & GIFTS,ALABAMA,CAMDEN,LAWRENCEVILLE,None,None,S8HPNTSCZA,-117.827848,"REAL PROPERTY LESSORS, NEC",None,None,None,0,"AUTOMOBILE PARKING, NSK",None,None,None,None,14,00000,None,00,None,USA,DIVISION G. - RETAIL TRADE,GU,HIGH,None,None,GU,None,None,GENERAL CLOTHING AND ACCESSORIES,721214,None,None,001,None,None,None,"6000 EXECUTIVE BLVD # 700, ROCKVILLE, MARYLAND...",None,None,208522783,None,None,None,N,None,None,None,None,None,None,None
1,None,USA,CHALAN DAOG,None,None,53310000,ORGANIZATION HOTELS AND LODGING HOUSES ON MEMB...,None,"EXECUTIVE, LEGISLATIVE AND GENERAL GOVERNMENT,...",1122858373,IN,2041145109#144.6722222#13.3155556,None,10050191,(912) 275-7393,2020-08-01,33.704969,None,None,NICKY NICOLE,WYOMING,ORANGE,BAKERSFIELD,None,None,S8HPNTSCZA,-96.137244,"REAL PROPERTY LESSORS, NEC",None,FRANKLIN PUBLIC WORKS DEPT,None,0,"AUTOMOBILE PARKING, NSK",None,None,None,None,60,00000,None,00,None,USA,DIVISION G. - RETAIL TRADE,GU,HIGH,None,None,GU,None,None,GENERAL CLOTHING AND ACCESSORIES,721110,None,None,001,None,None,111 MIDDLETON RD,"1101 ELLER DR, FORT LAUDERDALE, FLORIDA, 33316...",None,None,None,None,None,None,N,None,None,None,None,None,None,None
2,WWW.COTTLEFARMS.COM,USA,AJAYAN,None,None,53310000,CAMPS AND RECREATIONAL VEHICLE PARKS,None,ADMINISTRATION OF HUMAN RESOURCE PROGRAMS,1129857835,MI,2041107895#144.7163889#13.2494444,None,10050191,(936) 876-4864,2020-08-01,40.661038,None,None,NAVA'S DESIGN,CALIFORNIA,SALT LAKE,MOUNT HOPE,None,None,S8HPNTSCZA,-121.553364,"REAL PROPERTY LESSORS, NEC",None,None,None,1,"AUTOMOBILE PARKING, NSK",None,None,None,None,20,00009,None,01,None,None,DIVISION G. - RETAIL TRADE,GU,HIGH,None,283418534,GU,None,None,GENERAL CLOTHING AND ACCESSORIES,721214,None,None,001,None,None,744 S WEBSTER AVE,None,None,None,None,"2488 W NC 403 HWY, FAISON, NORTH CAROLINA, 283...",None,None,N,None,None,None,None,None,None,None
3,WWW.NESSRALLAFARMWAREHAM.COM,USA,GUGAGON,None,None,53310000,ORGANIZATION HOTELS AND LODGING HOUSES ON MEMB...,None,NATIONAL SECURITY AND INTERNATIONAL AFFAIRS,1251234950,LA,2041107894#144.7338889#13.28,None,10050191,(928) 763-1777,2020-08-01,33.549496,None,None,D & J KIDS FOUNDATION,CALIFORNIA,LUBBOCK,WASHINGTON,None,None,S8HPNTSCZA,-96.872766,"REAL PROPERTY LESSORS, NEC",None,FMS 13-FOREMAN,None,0,"AUTOMOBILE PARKING, NSK",None,None,None,None,15,00000,None,00,None,None,DIVISION G. - RETAIL TRADE,GU,HIGH,None,None,GU,None,None,GENERAL CLOTHING AND ACCESSORIES,721310,None,None,001,None,None,5100 POPLAR AVE STE 1212,"222 3RD AVE SE STE 285, CEDAR RAPIDS, IOWA, 52...",None,None,None,None,None,None,N,None,None,None,None,None,None,None
4,None,USA,ADAIQ

Let's now download a small sample to help us identify which of the four hierarchy variables gives us the pharmacies.

In [7]:
sql_query = "SELECT * except(do_label) FROM $dataset$ WHERE ST_IntersectsBox(geom, -75.161723,39.962019,-75.149535,39.968071)"
sample = dataset.to_dataframe(sql_query=sql_query)

In [8]:
sample.head()

,geoid,do_date,NAME,BRANDNAME,PB_ID,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,STABB,POSTCODE,FORMATTEDADDRESS,MAINADDRESSLINE,ADDRESSLASTLINE,LONGITUDE,LATITUDE,GEORESULT,CONFIDENCE_CODE,COUNTRY_ACCESS_CODE,TEL_NUM,FAXNUM,EMAIL,HTTP,OPEN_24H,BUSINESS_LINE,SIC1,SIC2,SIC8,SIC8_DESCRIPTION,ALT_INDUSTRY_CODE,MICODE,TRADE_DIVISION,GROUP,CLASS,SUB_CLASS,EMPLOYEE_HERE,EMPLOYEE_COUNT,YEAR_START,SALES_VOLUME_LOCAL,SALES_VOLUME_US_DOLLARS,CURRENCY_CODE,AGENT_CODE,LEGAL_STATUS_CODE,STATUS_CODE,SUBSIDIARY_INDICATOR,PARENT_BUSINESS_NAME,PARENT_ADDRESS,PARENT_STREET_ADDRESS,PARENT_AREANAME3,PARENT_AREANAME1,PARENT_COUNTRY,PARENT_POSTCODE,DOMESTIC_ULTIMATE_BUSINESS_NAME,DOMESTIC_ULTIMATE_ADDRESS,DOMESTIC_ULTIMATE_STREET_ADDRESS,DOMESTIC_ULTIMATE_AREANAME3,DOMESTIC_ULTIMATE_AREANAME1,DOMESTIC_ULTIMATE_POSTCODE,GLOBAL_ULTIMATE_INDICATOR,GLOBAL_ULTIMATE_BUSINESS_NAME,GLOBAL_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_STREET_ADDRESS,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom
0,2019661767#-75.160361#39.96587,2019-10-01,DIVING BELL SCUBA SHOP,NaN,2019661767,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19123,"681 N BROAD ST, PHILADELPHIA, PA, 19123",681 N BROAD ST,"PHILADELPHIA, PA, 19123",-75.160361,39.965870,S8HPNTSCZA,HIGH,NaN,(215) 763-6868,NaN,HOWARD_PRUYN@DIVINGBELL.COM,NaN,NaN,NaN,NaN,NaN,NaN,WATER SPORT,NaN,10120300,DIVISION M. - SPORTS,SPORTS,WATER SPORT,WATER SPORT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.160361 39.96587)
1,2019661767#-75.160361#39.96587,2019-10-01,DIVING BELL SCUBA SHOP,NaN,2019661767,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19123,"681 N BROAD ST, PHILADELPHIA, PA, 19123",681 N BROAD ST,"PHILADELPHIA, PA, 19123",-75.160361,39.965870,S8HPNTSCZA,HIGH,NaN,(215) 763-6868,NaN,HOWARD_PRUYN@DIVINGBELL.COM,NaN,NaN,NaN,NaN,NaN,NaN,WATER SPORT,NaN,10120300,DIVISION M. - SPORTS,SPORTS,WATER SPORT,WATER SPORT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.160361 39.96587)
2,2039673275#-75.15423#39.963883,2020-02-01,ST PAULS FCU,NaN,2039673275,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19123,"1000 WALLACE ST, PHILADELPHIA, PA, 19123",1000 WALLACE ST,"PHILADELPHIA, PA, 19123",-75.154230,39.963883,S7HPNTSCZA,HIGH,NaN,(215) 769-5199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BANK,NaN,10030310,"DIVISION H. - FINANCE, INSURANCE, AND REAL ESTATE",DEPOSITORY INSTITUTIONS,BANK,BANK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.15423 39.963883)
3,2155569554#-75.15666#39.965252,2020-02-01,TRANSIT WORKERS FCU,NaN,2155569554,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19123,"1204 MELON ST, PHILADELPHIA, PA, 19123",1204 MELON ST,"PHILADELPHIA, PA, 19123",-75.156660,39.965252,S7HPNTSCZA,HIGH,NaN,(215) 769-3899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BANK,NaN,10030310,"DIVISION H. - FINANCE, INSURANCE, AND REAL ESTATE",DEPOSITORY INSTITUTIONS,BANK,BANK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.15666 39.965252)
4,1116489911#-75.157314#39.962857,2019-09-01,"DRINKER DOWNS, INC.",NaN,1116489911,NaN,NaN,USA,NaN,PHILADELPHIA,NaN,PENNSYLVANIA,PA,191233332,"549 N 12TH ST, PHILADELPHIA, PA 191233332",549 N 12TH ST,"PHILADELPHIA, PA 191233332",-75.157314,39.962857,S8HPNTSCZA,HIGH,1.0,(267) 318-7772,NaN,NaN,WWW.INSTITUTEBAR.COM/,NaN,"DRINKING PLACES, NSK",5813.0,NaN,58130200.0,NIGHT CLUBS,722410.0,10130261,DIVISION G. - RETAIL TRADE,EATING AND DRINKING PLACES,DRINKING PLACES,"DRINKING PLACES/CAFE, PUB",4.0,4.0,2008.0,165700.0,165700.0,20.0,G,3.0,0.0,0.0,NaN,NaN,NaN,Na

In [9]:
sample['TRADE_DIVISION'].unique()

array(['DIVISION M. - SPORTS',
       'DIVISION H. - FINANCE, INSURANCE, AND REAL ESTATE',
       'DIVISION G. - RETAIL TRADE', 'DIVISION I. - SERVICES',
       'DIVISION C. - CONSTRUCTION',
       'DIVISION J. - PUBLIC ADMINISTRATION',
       'DIVISION E. - TRANSPORTATION AND PUBLIC UTILITIES',
       'DIVISION L. - TOURISM'], dtype=object)

In [10]:
sample.loc[sample['TRADE_DIVISION'] == 'DIVISION G. - RETAIL TRADE', 'GROUP'].unique()

array(['EATING AND DRINKING PLACES',
       'BUILDING MATERIALS, HARDWARE, GARDEN SUPPLIES AND MOBILE HOMES',
       'APPAREL AND ACCESSORY STORES',
       'AUTOMOTIVE DEALERS AND GASOLINE SERVICE STATIONS',
       'MISCELLANEOUS RETAIL',
       'HOME FURNITURE, FURNISHINGS AND EQUIPMENT STORES', 'FOOD STORES'],
      dtype=object)

In [11]:
sample.loc[sample['TRADE_DIVISION'] == 'DIVISION G. - RETAIL TRADE', 'CLASS'].unique()

array(['DRINKING PLACES', 'PAINT, GLASS, AND WALLPAPER STORES',
       'MISCELLANEOUS APPAREL AND ACCESSORY STORES',
       'AUTO AND HOME SUPPLY STORES',
       'MISCELLANEOUS SHOPPING GOODS STORES',
       'HOME FURNITURE AND FURNISHINGS STORES',
       'DRUG STORES AND PROPRIETARY STORES', 'LIQUOR STORES',
       'MOTOR VEHICLE DEALERS (NEW AND USED)',
       'GASOLINE SERVICE STATIONS', 'EATING PLACES/RESTAURANTS',
       'FAMILY CLOTHING STORES',
       'RETAIL STORES, NOT ELSEWHERE CLASSIFIED', 'NONSTORE RETAILERS',
       'GROCERY STORES', 'FRUIT AND VEGETABLE MARKETS',
       'LUMBER AND OTHER BUILDING MATERIALS DEALERS'], dtype=object)

The class `DRUG STORES AND PROPRIETARY STORES` is the one we're looking for.

In [12]:
sample.loc[sample['CLASS'] == 'DRUG STORES AND PROPRIETARY STORES', 'SUB_CLASS'].unique()

array(['DRUG STORES AND PROPRIETARY STORES/PHARMACY'], dtype=object)

### Download all pharmacies in the area of study

In [13]:
sql_query = """SELECT * except(do_label) 
               FROM $dataset$
               WHERE CLASS = 'DRUG STORES AND PROPRIETARY STORES'
               AND ST_IntersectsBox(geom, -75.229353,39.885501,-75.061124,39.997898)"""

ph_pharmacies = dataset.to_dataframe(sql_query=sql_query)

In [14]:
ph_pharmacies.head()

,geoid,do_date,NAME,BRANDNAME,PB_ID,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,STABB,POSTCODE,FORMATTEDADDRESS,MAINADDRESSLINE,ADDRESSLASTLINE,LONGITUDE,LATITUDE,GEORESULT,CONFIDENCE_CODE,COUNTRY_ACCESS_CODE,TEL_NUM,FAXNUM,EMAIL,HTTP,OPEN_24H,BUSINESS_LINE,SIC1,SIC2,SIC8,SIC8_DESCRIPTION,ALT_INDUSTRY_CODE,MICODE,TRADE_DIVISION,GROUP,CLASS,SUB_CLASS,EMPLOYEE_HERE,EMPLOYEE_COUNT,YEAR_START,SALES_VOLUME_LOCAL,SALES_VOLUME_US_DOLLARS,CURRENCY_CODE,AGENT_CODE,LEGAL_STATUS_CODE,STATUS_CODE,SUBSIDIARY_INDICATOR,PARENT_BUSINESS_NAME,PARENT_ADDRESS,PARENT_STREET_ADDRESS,PARENT_AREANAME3,PARENT_AREANAME1,PARENT_COUNTRY,PARENT_POSTCODE,DOMESTIC_ULTIMATE_BUSINESS_NAME,DOMESTIC_ULTIMATE_ADDRESS,DOMESTIC_ULTIMATE_STREET_ADDRESS,DOMESTIC_ULTIMATE_AREANAME3,DOMESTIC_ULTIMATE_AREANAME1,DOMESTIC_ULTIMATE_POSTCODE,GLOBAL_ULTIMATE_INDICATOR,GLOBAL_ULTIMATE_BUSINESS_NAME,GLOBAL_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_STREET_ADDRESS,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom
0,2154275062#-75.15871#39.935868,2020-01-01,HERBALIFE,NaN,2154275062,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19147,"1116 S 9TH ST, PHILADELPHIA, PA, 19147",1116 S 9TH ST,"PHILADELPHIA, PA, 19147",-75.158710,39.935868,S7HPNTSCZA,HIGH,NaN,(267) 519-3122,NaN,NaN,WWW.HERBALIFE.COM,NaN,NaN,5912,NaN,59129901,DRUG STORES,NaN,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.15871 39.935868)
1,2128682600#-75.156071#39.953738,2020-01-01,HERBALIFE,NaN,2128682600,NaN,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19107,"933 ARCH ST, PHILADELPHIA, PA, 19107",933 ARCH ST,"PHILADELPHIA, PA, 19107",-75.156071,39.953738,S8HPNTSCZA,HIGH,NaN,(267) 687-8065,NaN,NaN,WWW.HERBALIFE.COM,NaN,NaN,5912,NaN,59129901,DRUG STORES,NaN,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.156071 39.953738)
2,2038322702#-75.063221#39.946545,2020-01-01,SHAKLEE DISTRIBUTOR,NaN,2038322702,NaN,NaN,USA,NaN,MERCHANTVILLE,CAMDEN,NEW JERSEY,NJ,08109,"226 POPLAR AVE, MERCHANTVILLE, NJ, 08109",226 POPLAR AVE,"MERCHANTVILLE, NJ, 08109",-75.063221,39.946545,S7HPNTSCZA,HIGH,NaN,(856) 767-9203,NaN,NaN,WWW.SHAKLEE.COM,NaN,NaN,5912,NaN,59129901,DRUG STORES,NaN,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT(-75.063221 39.946545)
3,1130938998#-75.131713#39.988179,2020-01-01,KIM DO INC,NaN,1130938998,MR PHARMACY,NaN,USA,NaN,PHILADELPHIA,NaN,PENNSYLVANIA,PA,191334004,"2510 N FRONT ST, PHILADELPHIA, PA, 191334004",2510 N FRONT ST,"PHILADELPHIA, PA, 191334004",-75.131713,39.988179,S8HPNTSCZA,HIGH,1.0,(215) 634-3939,NaN,NaN,NaN,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59129901,DRUG STORES,446110.0,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,2.0,2.0,1989.0,154609.0,154609.0,20.0,G,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,POINT(-75.131713 39.988179)
4,1192983543#-75.13246#39.984668,2020-01-01,"FRIENDLY PHARMACY, INC.",NaN,1192983543,NaN,NaN,USA,NaN,PHILADELPHIA,NaN,PENNSYLVANIA,PA,191333714,"2258 N FRONT ST, PHILADELPHIA, PA, 191333714",2258 N FRONT ST,"PHILADELPHIA, PA, 191333714",-75.132460,39.984668,S8HPNTSCZA,HI

The dataset contains different versions of the POI's tagged by the _do_date_ column. We are only inetrested in the latest version of each POI.

In [15]:
ph_pharmacies = ph_pharmacies.sort_values(by='do_date', ascending=False).groupby('PB_ID').first().reset_index()

In [16]:
ph_pharmacies.shape

(477, 74)

### Visualize the dataset

In [17]:
Layer(ph_pharmacies, 
      geom_col='geom', 
      style=basic_style(opacity=0.75), 
      popup_hover=popup_element('NAME'))

<a id='section2'></a>
## 2. Calculate catchment areas

In order to know the characteristics of the potential customers of every pharmacy, we assume the majority of their clients live closeby. Therefore we will calculate **5-minute-by-car isochrones** and take them as their cathment areas.

_Note_ catchment areas usually depend on whether it is a store in the downtown area or in the suburbs, or if it is reachable on foot or only by car. For this example, we will not make such distiction between pharmacies, but we strongly encourage you to do so on your analyses. As an example, [here](https://carto.com/blog/calculating-catchment-human-mobility-data/) we describe how to calculate catchment areas using human mobility data.

In [18]:
iso_service = Isolines()
isochrones_gdf, _ = iso_service.isochrones(ph_pharmacies, [300], mode='car', geom_col='geom')

Success! Isolines created correctly


In [19]:
ph_pharmacies['iso_5car'] = isochrones_gdf.sort_values(by='source_id')['the_geom'].values

### Visualize isochrones

We'll only visualize the ten first isochrones to get a clean visualization.

In [20]:
Map([Layer(ph_pharmacies.iloc[:10], 
           geom_col='iso_5car', 
           style=basic_style(opacity=0.1),
           legends=basic_legend('Catchment Areas')),
     Layer(ph_pharmacies.iloc[:10],
           geom_col='geom',
           popup_hover=popup_element('NAME'),
           legends=basic_legend('Pharmacies'))])

<a id='section3'></a>
## 3. Enrichment: Chacacterize catchment areas

We'll now enrich the pharmacies catchment areas with demographics, POI's, and consumer spending data.

For the enrichment, we will use the CARTOframes Enrichment class. This class contains the functionality to enrich polygons and points. 

Visit [CARTOframes Guides](https://carto.com/developers/cartoframes/guides/) for further detail.

In [21]:
enrichment = Enrichment()

### Demographics

We will use AGS premium data. In particular, we will work with the dataset `ags_sociodemogr_f510a947` which contains [yearly demographics data from 2019](https://carto.com/spatial-data-catalog/browser/dataset/ags_sociodemogr_f510a947/).

#### Variable selection

Here we will enrich the pharmacies isochrones with:
 - Population aged 60+
 - Household income
 - Household income for population ages 65+

In [22]:
Catalog().country('usa').category('demographics').provider('ags').datasets.to_dataframe().head()

,slug,name,description,category_id,country_id,data_source_id,provider_id,geography_name,geography_description,temporal_aggregation,time_coverage,update_frequency,is_public_data,lang,version,category_name,provider_name,geography_id,id
0,ags_consumer_pr_9f337eb8,Consumer Profiles - United States of America (...,Segmentation of the population in sixty-eight ...,demographics,usa,consumer_profiles,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2018-01-01, 2019-01-01)",yearly,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_consumerprofiles_usa...
1,ags_sociodemogr_f510a947,Sociodemographics - United States of America (...,Census and ACS sociodemographic data estimated...,demographics,usa,sociodemographics,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2019-01-01, 2020-01-01)",yearly,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_sociodemographics_us...
2,ags_businesscou_df363a87,Business Counts - United States of America (Ce...,Business Counts database is a geographic summa...,demographics,usa,businesscounts,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2018-01-01, 2019-01-01)",yearly,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_businesscounts_usa_b...
3,ags_consumer_sp_895a369c,Consumer Spending - United States of America (...,The Consumer Expenditure database consists of ...,demographics,usa,consumer_spending,ags,Census Block Group - United States of America ...,Shoreline clipped TIGER/Line boundaries. More ...,yearly,"[2018-01-01, 2019-01-01)",yearly,False,eng,2019,Demographics,Applied Geographic Solutions,carto-do-public-data.carto.geography_usa_block...,carto-do.ags.demographics_consumerspending_usa...
4,ags_consumer_pr_cdc295df,Consumer Profiles - United States of America (...,Segmentation of the population in sixty-eight ...,demographics,usa,consumer_profiles,ags,Census Block Group - United States of America,None,yearly,None,yearly,False,eng,2020,Demographics,Applied Geographic Solutions,carto-do.ags.geography_usa_blockgroup_2015,carto-do.ags.demographics_consumerprofiles_usa...


In [23]:
dataset = Dataset.get('ags_sociodemogr_f510a947')

In [24]:
dataset.head()

,DWLCY,HHDCY,POPCY,VPHCY1,AGECYMED,HHDCYFAM,HOOEXMED,HUSEXAPT,LBFCYARM,LBFCYLBF,LBFCYNLF,MARCYSEP,POPCYGRP,RNTEXMED,SEXCYFEM,SEXCYMAL,VPHCYGT1,AGECY0004,AGECY0509,AGECY1014,AGECY1519,AGECY2024,AGECY2529,AGECY3034,AGECY3539,AGECY4044,AGECY4549,AGECY5054,AGECY5559,AGECY6064,AGECY6569,AGECY7074,AGECY7579,AGECY8084,AGECYGT15,AGECYGT25,AGECYGT85,DWLCYRENT,EDUCYBACH,EDUCYGRAD,EDUCYHSCH,HINCY1015,HINCY1520,HINCY2025,HINCY2530,HINCY3035,HINCY3540,HINCY4045,HINCY4550,HINCY5060,HINCY6075,HINCYLT10,HISCYHISP,HUSEX1DET,INCCYPCAP,LBFCYEMPL,LBFCYUNEM,LNIEXISOL,LNIEXSPAN,MARCYMARR,POPCYGRPI,UNECYRATE,VPHCYNONE,BLOCKGROUP,DWLCYOWNED,DWLCYVACNT,EDUCYASSOC,EDUCYLTGR9,EDUCYSCOLL,EDUCYSHSCH,HHDCYAVESZ,HHDCYMEDAG,HHSCYLPFCH,HHSCYLPMCH,HHSCYMCFCH,HINCY10025,HINCY12550,HINCY15020,HINCY75100,HINCYGT200,HINCYMED24,HINCYMED25,HINCYMED35,HINCYMED45,HINCYMED55,HINCYMED65,HINCYMED75,INCCYAVEHH,INCCYMEDFA,INCCYMEDHH,LBFCYPOP16,MARCYDIVOR,MARCYNEVER,MARCYWIDOW,RCHCYAMNHS,RCHCYASNHS,RCHCYBLNHS,RCHCYHANHS,RCHCYMUNHS,RCHCYOTNHS,RCHCYWHNHS
0,5,5,6,0,64.00,1,63749,0,0,0,6,0,0,0,5,1,5,0,0,0,0,0,0,0,0,0,1,0,1,1,2,0,0,0,6,6,1,0,0,0,5,0,0,0,0,0,0,0,0,0,5,0,0,2,53754,0,0,0,0,6,0,0.00,0,010159819011,5,0,0,1,0,0,1.2,61.5,0,0,0,0,0,0,0,0,0,0,0,67500,67500,67500,67500,64504,67499,67499,6,0,0,0,0,0,0,0,0,0,6
1,2,2,5,1,36.50,2,124999,0,0,2,1,0,0,0,3,2,1,0,0,2,0,0,0,0,1,0,0,0,0,2,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,33026,2,0,0,0,2,0,0.00,0,010159819021,2,0,0,1,2,0,2.5,54.0,1,0,0,0,0,0,2,0,0,0,87500,0,87500,0,0,82566,87499,87499,3,0,1,0,0,0,3,0,0,0,2
2,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,010159819031,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,21,11,22,4,64.00,6,74999,0,0,10,10,1,0,449,9,13,5,0,1,1,1,1,0,0,1,0,3,1,2,0,7,1,1,0,20,18,2,6,1,1,8,1,1,1,1,1,0,0,0,2,2,2,0,9,16647,10,0,0,0,0,0,0.00,2,010970004011,5,10,1,0,3,4,2.0,61.5,1,0,0,0,0,0,0,0,0,0,5000,60000,30000,50000,12500,33294,49999,27499,20,4,13,2,0,0,22,0,0,0,0
4,0,0,959,0,18.91,0,0,0,0,378,581,0,959,0,585,374,0,0,0,0,488,430,31,5,0,0,3,0,0,0,0,0,0,0,959,41,2,0,7,3,14,0,0,0,0,0,0,0,0,0,0,0,36,0,3753,346,32,0,0,0,0,8.47,0,010970036051,0,0,3,0,10,4,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,959,0,959,0,5,53,230,0,25,0,609
5,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,020900018001,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,040030014021,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,040030014022,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,040129800001,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,040139801001,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We explore the variables to identify the ones we're interested in. 

Variables in a dataset are uniquely identified by their slug.

In [25]:
dataset.variables.to_dataframe().head()

,slug,name,description,db_type,agg_method,column_name,variable_group_id,dataset_id,id
0,BLOCKGROUP_108673f9,BLOCKGROUP,Geographic Identifier,STRING,None,BLOCKGROUP,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
1,POPCY_5e23b8f4,Total Population,Population (2019A),INTEGER,SUM,POPCY,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
2,POPCYGRP_55c4a2e5,POPCYGRP,Population in Group Quarters (2019A),INTEGER,SUM,POPCYGRP,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
3,POPCYGRPI_d37c4ec,POPCYGRPI,Institutional Group Quarters Population (2019A),INTEGER,SUM,POPCYGRPI,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
4,AGECY0004_a67ddb4f,AGECY0004,Population age 0-4 (2019A),INTEGER,SUM,AGECY0004,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...


We'll select:
 - Population and population by age variables to identify number of people aged 60+ as a percentage of total population
 - Average household income
 - Average household income for porpulation aged 65+

In [26]:
vars_enrichment = ['POPCY_5e23b8f4', 'AGECY6064_d54c2315', 'AGECY6569_ad369d43', 'AGECY7074_74eb7531',
                   'AGECY7579_c91cb67', 'AGECY8084_ab1079a8', 'AGECYGT85_a0959a08', 'INCCYMEDHH_b80a7a7b',
                   'HINCYMED65_37a430a4', 'HINCYMED75_2ebf01e5']

#### Isochrone enrichment

In [27]:
ph_pharmacies_enriched = enrichment.enrich_polygons(
    ph_pharmacies,
    variables=vars_enrichment,
    geom_col='iso_5car'
)

In [28]:
ph_pharmacies_enriched.head()

,PB_ID,geoid,do_date,NAME,BRANDNAME,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,STABB,POSTCODE,FORMATTEDADDRESS,MAINADDRESSLINE,ADDRESSLASTLINE,LONGITUDE,LATITUDE,GEORESULT,CONFIDENCE_CODE,COUNTRY_ACCESS_CODE,TEL_NUM,FAXNUM,EMAIL,HTTP,OPEN_24H,BUSINESS_LINE,SIC1,SIC2,SIC8,SIC8_DESCRIPTION,ALT_INDUSTRY_CODE,MICODE,TRADE_DIVISION,GROUP,CLASS,SUB_CLASS,EMPLOYEE_HERE,EMPLOYEE_COUNT,YEAR_START,SALES_VOLUME_LOCAL,SALES_VOLUME_US_DOLLARS,CURRENCY_CODE,AGENT_CODE,LEGAL_STATUS_CODE,STATUS_CODE,SUBSIDIARY_INDICATOR,PARENT_BUSINESS_NAME,PARENT_ADDRESS,PARENT_STREET_ADDRESS,PARENT_AREANAME3,PARENT_AREANAME1,PARENT_COUNTRY,PARENT_POSTCODE,DOMESTIC_ULTIMATE_BUSINESS_NAME,DOMESTIC_ULTIMATE_ADDRESS,DOMESTIC_ULTIMATE_STREET_ADDRESS,DOMESTIC_ULTIMATE_AREANAME3,DOMESTIC_ULTIMATE_AREANAME1,DOMESTIC_ULTIMATE_POSTCODE,GLOBAL_ULTIMATE_INDICATOR,GLOBAL_ULTIMATE_BUSINESS_NAME,GLOBAL_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_STREET_ADDRESS,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom,iso_5car,POPCY,AGECY8084,AGECYGT85,AGECY6569,AGECY7579,AGECY7074,AGECY6064,HINCYMED65,HINCYMED75,INCCYMEDHH
0,1114689327,1114689327#-75.186601#39.918301,2020-08-01,"WALGREEN EASTERN CO., INC.",WALGREENS,WALGREENS,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19145-4122,"2310 W OREGON AVE, PHILADELPHIA, PA, 19145-4122",2310 W OREGON AVE,"PHILADELPHIA, PA, 19145-4122",-75.186601,39.918301,S8HPNTSCZA,HIGH,1.0,(215) 468-2481,NaN,NaN,WWW.WALLGREENSBOOTSALLIANCE.COM,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59129901,DRUG STORES,446110.0,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,40.0,0.0,0.0,0.0,0.0,NaN,G,0.0,2.0,0.0,"WALGREEN EASTERN CO., INC.","200 WILMOT RD, DEERFIELD, ILLINOIS, 600154620,...",200 WILMOT RD,DEERFIELD,ILLINOIS,USA,600154620.0,"WALGREENS BOOTS ALLIANCE, INC.","108 WILMOT RD, DEERFIELD, ILLINOIS, 600155145",108 WILMOT RD,DEERFIELD,ILLINOIS,600155145.0,N,"WALGREENS BOOTS ALLIANCE, INC.","108 WILMOT RD, DEERFIELD, ILLINOIS, 600155145,...",108 WILMOT RD,DEERFIELD,ILLINOIS,USA,600155145.0,9551.0,3.0,NaN,NaN,POINT(-75.186601 39.918301),"MULTIPOLYGON (((-75.19996 39.93393, -75.19953 ...",15037.340234,293.522613,310.290651,783.930533,454.807673,599.859593,922.264170,50981.227273,38971.636364,55397.409091
1,1114942469,1114942469#-75.160179#39.970478,2020-08-01,TANG PHARMACY IV,NaN,TANG PHARMACY IV,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19130-1604,"900 N BROAD ST, PHILADELPHIA, PA, 19130-1604",900 N BROAD ST,"PHILADELPHIA, PA, 19130-1604",-75.160179,39.970478,S8HPNTSCZA,HIGH,1.0,(215) 235-1400,NaN,NaN,NaN,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59120000,DRUG STORES AND PROPRIETARY STORES,446110.0,10230030,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,4.0,4.0,2012.0,610000.0,610000.0,20.0,G,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,POINT(-75.160179 39.970478),"MULTIPOLYGON (((-75.17592 39.96826, -75.17550 ...",55923.104385,833.022141,1011.555847,2455.947041,1276.840519,1846.609283,2953.948643,45964.879310,37373.931034,49339.293103
2,1115024609,1115024609#-75.067035#39.935705,2020-08-01,AMERICARE 1 PHARMACY,NaN,NaN,NaN,USA,NaN,MERCHANTVILLE,CAMDEN,NEW JERSEY,NJ,08109-4803,"5115 ROUTE 38, MERCHANTVILLE, NJ, 08109-4803",5115 ROUTE 38,"MERCHANTVILLE, NJ, 08109-4803",-75.067035,39.935705,S8HPNTSCZA,HIGH,1.0,(215) 821-2720,NaN,NaN,WWW.ACAREPHARMACY.COM,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59129901,DRUG STORES,446110.0,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,5.0,5.0,2011.0,616316.0,616316.0,20.0,G,13.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Na

In [29]:
ph_pharmacies = ph_pharmacies_enriched.copy()

In [30]:
ph_pharmacies['pop_60plus'] = ph_pharmacies[['AGECY8084', 'AGECYGT85', 'AGECY6569', 'AGECY7579', 'AGECY7074', 'AGECY6064']].sum(1)
ph_pharmacies.drop(columns=['AGECY8084', 'AGECYGT85', 'AGECY6569', 'AGECY7579', 'AGECY7074', 'AGECY6064'], inplace=True)

### Points of Interest

We will use [Pitney Bowes' Consumer Points of Interest](https://carto.com/spatial-data-catalog/browser/dataset/pb_consumer_po_62cddc04/) premium dataset.

#### Variable selection

We are interested in knowing how many of the following POIs can be found in each isochrone:
 - Beauty shops and beauty salons
 - Gyms and other sports centers

These POI's will be considered as an indicator of personal care awareness in a specific area.

The hierarchy classification variable `SUB_CLASS` variable allows us to identify beaty shops and salons (`BEAUTY SHOPS/BEAUTY SALON`) and gyms (`MEMBERSHIP SPORTS AND RECREATION CLUBS/CLUB AND ASSOCIATION - UNSPECIFIED`).

In [31]:
sample.loc[sample['TRADE_DIVISION'] == 'DIVISION I. - SERVICES', 'SUB_CLASS'].unique()

array(['LABOR ORGANIZATIONS', 'MEMBERSHIP ORGANIZATIONS, NEC',
       'MEDIA FACILITY', 'COLLEGES AND UNIVERSITIES',
       'OFFICES AND CLINICS OF DENTISTS/ DENTIST',
       'RELIGIOUS ORGANIZATIONS', 'CIVIC AND SOCIAL ASSOCIATIONS',
       'FLATS, APARTMENT COMPLEX', 'HOTEL, MOTEL - UNSPECIFIED',
       'CARWASHES', 'OFFICES AND CLINICS OF MEDICAL DOCTORS',
       'PARK AND RECREATION AREA - UNSPECIFIED',
       'SKILLED NURSING CARE FACILITIES',
       'NURSING AND PERSONAL CARE, NEC',
       'SPECIALTY OUTPATIENT CLINICS, NEC',
       'HEALTH AND ALLIED SERVICES, NEC',
       'TOP AND BODY REPAIR AND PAINT SHOPS/BODYSHOPS',
       'GENERAL AUTOMOTIVE REPAIR SHOPS/GENERAL CAR REPAIR AND SERVICING',
       'VOCATIONAL SCHOOLS, NEC/VOCATIONAL TRAINING', 'LIBRARIES',
       'SCHOOL', 'GENERAL MEDICAL AND SURGICAL HOSPITALS',
       'SCHOOLS AND EDUCATIONAL SERVICES/SCHOOL - UNSPECIFIED',
       'RETIREMENT COMMUNITY', 'OFFICES OF HEALTH PRACTITIONER',
       'BARBER SHOPS/HAIRDRESSERS 

#### Isochrone enrichment

In order to count only Beauty Shops/Salons and Gyms, we will apply a filter to the enrichment. All filters are applied with an AND-like relationship. This means we need to run two independent enrichment calls, one for the beauty shops/salons and another one for the gyms.

In [32]:
ph_pharmacies_enriched = enrichment.enrich_polygons(
    ph_pharmacies,
    variables=['SUB_CLASS_10243439'],
    aggregation='COUNT',
    geom_col='iso_5car',
    filters={Variable.get('SUB_CLASS_10243439').id : "= 'BEAUTY SHOPS/BEAUTY SALON'"}
)

In [33]:
ph_pharmacies = ph_pharmacies_enriched.rename(columns={'SUB_CLASS_y':'n_beauty_pois'})

In [34]:
ph_pharmacies_enriched = enrichment.enrich_polygons(
    ph_pharmacies,
    variables=['SUB_CLASS_10243439'],
    aggregation='COUNT',
    geom_col='iso_5car',
    filters={Variable.get('SUB_CLASS_10243439').id : "= 'MEMBERSHIP SPORTS AND RECREATION CLUBS/CLUB AND ASSOCIATION - UNSPECIFIED'"}
)

In [35]:
ph_pharmacies = ph_pharmacies_enriched.rename(columns={'SUB_CLASS':'n_gym_pois'})

In [36]:
ph_pharmacies['n_pois_personal_care'] = ph_pharmacies['n_beauty_pois'] + ph_pharmacies['n_gym_pois']
ph_pharmacies.drop(columns=['n_beauty_pois', 'n_gym_pois'], inplace=True)

### Consumer spending

For consumer spending, we will use AGS premium data. In particular, we will work with the dataset `ags_consumer_sp_dbabddfb` which contains the [latest version of yearly consumer data](https://carto.com/spatial-data-catalog/browser/dataset/ags_consumer_sp_dbabddfb/).

#### Variable selection

We are interested in spending in:
 - Personal care services
 - Personal care products
 - Health care services

In [37]:
dataset = Dataset.get('ags_consumer_sp_dbabddfb')

In [38]:
dataset.variables.to_dataframe().head()

,slug,name,description,db_type,agg_method,column_name,variable_group_id,dataset_id,id
0,BLOCKGROUP_9c78947b,BLOCKGROUP,Geographic Identifier,STRING,None,BLOCKGROUP,None,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...
1,HHDCY_11b160d2,Number of households,Households (2019A),INTEGER,SUM,HHDCY,None,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...
2,XCYAP1_ef233ec0,XCYAP1,Men's Apparel (Ave Hhd Exp),FLOAT,AVG,XCYAP1,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...
3,XCYAP2_762a6f7a,XCYAP2,Boys Apparel (Ave Hhd Exp),FLOAT,AVG,XCYAP2,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...
4,XCYAP3_12d5fec,XCYAP3,Women's Apparel (Ave Hhd Exp),FLOAT,AVG,XCYAP3,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...,carto-do.ags.demographics_consumerspending_usa...


The variables we're interested in are:
 - `XCYHC2`  Health care services expenditure
 - `XCYPC3`  Personal care services expenditure
 - `XCYPC4`  Personal care products expenditure

In [39]:
Variable.get('XCYHC2_18141567').to_dict()

{'slug': 'XCYHC2_18141567',
 'name': 'XCYHC2',
 'description': 'Health Care Services (Ave Hhd Exp)',
 'db_type': 'FLOAT',
 'agg_method': 'AVG',
 'column_name': 'XCYHC2',
 'variable_group_id': 'carto-do.ags.demographics_consumerspending_usa_blockgroup_2015_yearly_2020.consumer_spending_by_product_category',
 'dataset_id': 'carto-do.ags.demographics_consumerspending_usa_blockgroup_2015_yearly_2020',
 'id': 'carto-do.ags.demographics_consumerspending_usa_blockgroup_2015_yearly_2020.XCYHC2'}

In [40]:
ph_pharmacies_enriched = enrichment.enrich_polygons(
    ph_pharmacies,
    variables=['XCYPC3_7d26d739', 'XCYPC4_e342429a', 'XCYHC2_18141567'],
    geom_col='iso_5car'
)

We rename the new columns to give them a more descriptive name.

In [41]:
ph_pharmacies = ph_pharmacies_enriched.rename(columns={'XCYHC2':'health_care_services_exp',
                                                       'XCYPC3':'personal_care_services_exp',
                                                       'XCYPC4':'personal_care_products_exp'})

In [42]:
ph_pharmacies.head(2)

,PB_ID,geoid,do_date,NAME,BRANDNAME,TRADE_NAME,FRANCHISE_NAME,ISO3,AREANAME4,AREANAME3,AREANAME2,AREANAME1,STABB,POSTCODE,FORMATTEDADDRESS,MAINADDRESSLINE,ADDRESSLASTLINE,LONGITUDE,LATITUDE,GEORESULT,CONFIDENCE_CODE,COUNTRY_ACCESS_CODE,TEL_NUM,FAXNUM,EMAIL,HTTP,OPEN_24H,BUSINESS_LINE,SIC1,SIC2,SIC8,SIC8_DESCRIPTION,ALT_INDUSTRY_CODE,MICODE,TRADE_DIVISION,GROUP,CLASS,SUB_CLASS_x,EMPLOYEE_HERE,EMPLOYEE_COUNT,YEAR_START,SALES_VOLUME_LOCAL,SALES_VOLUME_US_DOLLARS,CURRENCY_CODE,AGENT_CODE,LEGAL_STATUS_CODE,STATUS_CODE,SUBSIDIARY_INDICATOR,PARENT_BUSINESS_NAME,PARENT_ADDRESS,PARENT_STREET_ADDRESS,PARENT_AREANAME3,PARENT_AREANAME1,PARENT_COUNTRY,PARENT_POSTCODE,DOMESTIC_ULTIMATE_BUSINESS_NAME,DOMESTIC_ULTIMATE_ADDRESS,DOMESTIC_ULTIMATE_STREET_ADDRESS,DOMESTIC_ULTIMATE_AREANAME3,DOMESTIC_ULTIMATE_AREANAME1,DOMESTIC_ULTIMATE_POSTCODE,GLOBAL_ULTIMATE_INDICATOR,GLOBAL_ULTIMATE_BUSINESS_NAME,GLOBAL_ULTIMATE_ADDRESS,GLOBAL_ULTIMATE_STREET_ADDRESS,GLOBAL_ULTIMATE_AREANAME3,GLOBAL_ULTIMATE_AREANAME1,GLOBAL_ULTIMATE_COUNTRY,GLOBAL_ULTIMATE_POSTCODE,FAMILY_MEMBERS,HIERARCHY_CODE,TICKER_SYMBOL,EXCHANGE_NAME,geom,iso_5car,POPCY,HINCYMED65,HINCYMED75,INCCYMEDHH,pop_60plus,n_pois_personal_care,health_care_services_exp,personal_care_services_exp,personal_care_products_exp
0,1114689327,1114689327#-75.186601#39.918301,2020-08-01,"WALGREEN EASTERN CO., INC.",WALGREENS,WALGREENS,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19145-4122,"2310 W OREGON AVE, PHILADELPHIA, PA, 19145-4122",2310 W OREGON AVE,"PHILADELPHIA, PA, 19145-4122",-75.186601,39.918301,S8HPNTSCZA,HIGH,1.0,(215) 468-2481,NaN,NaN,WWW.WALLGREENSBOOTSALLIANCE.COM,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59129901,DRUG STORES,446110.0,10010700,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,40.0,0.0,0.0,0.0,0.0,NaN,G,0.0,2.0,0.0,"WALGREEN EASTERN CO., INC.","200 WILMOT RD, DEERFIELD, ILLINOIS, 600154620,...",200 WILMOT RD,DEERFIELD,ILLINOIS,USA,600154620.0,"WALGREENS BOOTS ALLIANCE, INC.","108 WILMOT RD, DEERFIELD, ILLINOIS, 600155145",108 WILMOT RD,DEERFIELD,ILLINOIS,600155145.0,N,"WALGREENS BOOTS ALLIANCE, INC.","108 WILMOT RD, DEERFIELD, ILLINOIS, 600155145,...",108 WILMOT RD,DEERFIELD,ILLINOIS,USA,600155145.0,9551.0,3.0,NaN,NaN,POINT(-75.186601 39.918301),"MULTIPOLYGON (((-75.19996 39.93393, -75.19953 ...",15037.340234,50981.227273,38971.636364,55397.409091,3364.675232,754.0,925.483182,349.088636,302.686364
1,1114942469,1114942469#-75.160179#39.970478,2020-08-01,TANG PHARMACY IV,NaN,TANG PHARMACY IV,NaN,USA,NaN,PHILADELPHIA,PHILADELPHIA,PENNSYLVANIA,PA,19130-1604,"900 N BROAD ST, PHILADELPHIA, PA, 19130-1604",900 N BROAD ST,"PHILADELPHIA, PA, 19130-1604",-75.160179,39.970478,S8HPNTSCZA,HIGH,1.0,(215) 235-1400,NaN,NaN,NaN,NaN,DRUG STORES AND PROPRIETARY STORES,5912,NaN,59120000,DRUG STORES AND PROPRIETARY STORES,446110.0,10230030,DIVISION G. - RETAIL TRADE,MISCELLANEOUS RETAIL,DRUG STORES AND PROPRIETARY STORES,DRUG STORES AND PROPRIETARY STORES/PHARMACY,4.0,4.0,2012.0,610000.0,610000.0,20.0,G,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,POINT(-75.160179 39.970478),"MULTIPOLYGON (((-75.17592 39.96826, -75.17550 ...",55923.104385,45964.879310,37373.931034,49339.293103,10377.923475,1874.0,828.155172,319.795172,277.296207


<a id='section4'></a>
## 4. Dashboard

Finally, with all the data gathered, we will build the dashboard and publish it so we can share it with our client/manager/colleague for them to explore it.

This dashboard allows you to select a range of desired expenditure in care products, people aged 60+, household income, and so forth. Selecting the desired ranges will filter out pharmacies, so that in the end you can identify the target pharmacies for your marketing campaign.

In [43]:
cmap = Map(Layer(ph_pharmacies, 
          geom_col='geom', 
          style=color_category_style('SIC8_DESCRIPTION', size=4, opacity=0.85, palette='safe', stroke_width=0.15), 
          widgets=[formula_widget(
                                    'PB_ID', 
                                    operation='COUNT', 
                                    title='Total number of pharmacies', 
                                    description='Keep track of the total amount of pharmacies that meet the ranges selected on the widgets below'),
                  histogram_widget(
                                    'pop_60plus',
                                    title='Population 60+',
                                    description='Select a range of values to filter',
                                    buckets=15
                                ),
                   histogram_widget(
                                    'HINCYMED65',
                                    title='Household income 65-74',
                                    buckets=15
                                ),
                   histogram_widget(
                                    'HINCYMED75',
                                    title='Household income 75+',
                                    buckets=15
                                ),
                  histogram_widget(
                                    'n_pois_personal_care',
                                    title='Number of personal care POIs',
                                    buckets=15
                                ),
                  histogram_widget(
                                    'personal_care_products_exp',
                                    title='Expenditure in personal care products ($)',
                                    buckets=15
                                )], 
          legends=color_category_legend(
                                        title='Pharmacies',
                                        description='Type of store'), 
          popup_hover=[popup_element('NAME', title='Name')]
    ), 
    viewport={'zoom': 11}
)

In [44]:
cmap

### Publish dashboard

In [ ]:
cmap.publish('ph_pharmacies_dashboard', password='MY_PASS')